# 4.10 Task Part 1.1 - Coding Etiquette and Excel Reporting

### Script contents:

#### 1. Import libraries and data
#### 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.
#### 3. Create a regional segmentation of the data
#### 4. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data
#### 5. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents

## Import libraries and data

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

#### 1. Import the data set you exported in Exercise 4.9 into a new Jupyter notebook. This should be the merged version of your data that contains the new customers column.


In [2]:
# Creating a string of the path to main project folder

path = '/Users/jarrettpugh/Library/CloudStorage/OneDrive-Personal/Data Analytics/Career Foundry - DA Bootcamp/A4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'

In [3]:
# Import customers_orders_products.pkl as dataframe 'cust_ords_prods'

cust_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_orders_products.pkl'))

In [4]:
# Import states_regions.csv as dataframe 'states_regions'

states_regions = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'states_regions.csv'))

In [5]:
# Import departments_wrangled.csv as dataframe 'departments'

departments = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'), index_col=None)

#### 2. Consider any security implications that might exist for this new data. You’ll need to address any PII data in the data before continuing your analysis.


In [6]:
cust_ords_prods[['first_name', 'last_name']]

,first_name,last_name
0,Linda,Nguyen
1,Linda,Nguyen
2,Linda,Nguyen
3,Linda,Nguyen
4,Linda,Nguyen
...,...,...
32404854,Gerald,Yates
32404855,Jacqueline,Arroyo
32404856,Jacqueline,Arroyo
32404857,Jacqueline,Arroyo


In [7]:
# Dropping 'first_name' and 'last_name' columns to ensure data privacy and security of PII data

cust_ords_prods = cust_ords_prods.drop(['first_name', 'last_name'], axis=1)

In [8]:
# Checking for any other columns with PII data

cust_ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state', 'customer_age',
       'date_joined', 'customer_number_of_dependents',
       'customer_family_status', 'customer_income', '_merge'],
      dtype='object')

In [9]:
cust_ords_prods = cust_ords_prods.drop('_merge', axis=1)

#### 3. The Instacart officers are interested in comparing customer behavior in different geographic areas. Create a regional segmentation of the data. You’ll need to create a “Region” column based on the “State” column from your customers data set.

    -Use the region information in this Wikipedia article to create your column (you only need to create regions, not divisions).
    
    -Determine whether there’s a difference in spending habits between the different U.S. regions. (Hint: You can do this by crossing the variable you just created with the spending flag.)

In [10]:
states_regions = states_regions.reset_index(drop=True)

In [11]:
# Rename columns to match cust_ords_prods organization

states_regions = states_regions.rename(columns={'State': 'customer_state', 'Region': 'customer_region'})

In [12]:
states_regions

,customer_state,customer_region
0,Maine,Northeast
1,New Hampshire,Northeast
2,Vermont,Northeast
3,Massachusetts,Northeast
4,Rhode Island,Northeast
5,Connecticut,Northeast
6,New York,Northeast
7,Pennsylvania,Northeast
8,New Jersey,Northeast
9,North Dakota,Midwest


In [13]:
# Merge states_regions df with cust_ords_prods df to create regional segmentation of the customer data

cust_ords_prods = cust_ords_prods.merge(states_regions, on='customer_state', how='left')

In [14]:
# Check frequency count of any NULL values or errors for new column 'customer_region'

cust_ords_prods['customer_region'].value_counts(dropna=False)

customer_region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [15]:
cust_ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state', 'customer_age',
       'date_joined', 'customer_number_of_dependents',
       'customer_family_status', 'customer_income', 'customer_region'],
      dtype='object')

In [16]:
# Reorder columns for customer state and customer region to be next to each other

cust_ords_prods = cust_ords_prods[['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state', 'customer_region', 'customer_age',
       'date_joined', 'customer_number_of_dependents',
       'customer_family_status', 'customer_income']]

In [17]:
cust_ords_prods.shape

(32404859, 31)

In [18]:
cust_ords_prods[['spending_flag', 'customer_region']].head()

,spending_flag,customer_region
0,Low Spender,South
1,Low Spender,South
2,Low Spender,South
3,Low Spender,South
4,Low Spender,South


In [19]:
regional_spending_crosstab = pd.crosstab(cust_ords_prods['spending_flag'], cust_ords_prods['customer_region'], dropna=False)

In [20]:
regional_spending_crosstab

customer_region,Midwest,Northeast,South,West
spending_flag,,,,
High Spender,155975,108225,209691,160354
Low Spender,7441350,5614511,10582194,8132559


#### 4. The Instacart CFO isn’t interested in customers who don’t generate much revenue for the app. Create an exclusion flag for low-activity customers (customers with less than 5 orders) and exclude them from the data. Make sure you export this sample.

In [21]:
# Create exclusion flag 'customer_activity' to find low-activity customers who have less than 5 orders

In [22]:
# Look at frequency count of customers' max order

cust_ords_prods['max_order'].value_counts(dropna=False)

max_order
99    1171333
8      811843
6      811396
9      810213
7      803979
       ...   
97      44949
98      44587
96      40453
2           6
1           5
Name: count, Length: 99, dtype: int64

In [23]:
# Create conditions

# If 'max_order' is 5 or higher, value in 'customer_activity' will be 'normal'

cust_ords_prods.loc[cust_ords_prods['max_order'] >= 5, 'customer_activity'] = 'normal'

/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/3431691107.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'normal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cust_ords_prods.loc[cust_ords_prods['max_order'] >= 5, 'customer_activity'] = 'normal'


In [24]:
# Create conditions

# If 'max_order' is less than 5, value in 'customer_activity' will be 'low'

cust_ords_prods.loc[cust_ords_prods['max_order'] < 5, 'customer_activity'] = 'low'

In [25]:
# Check value frequency for 'customer_activity' column

cust_ords_prods['customer_activity'].value_counts(dropna=False)

customer_activity
normal    30964564
low        1440295
Name: count, dtype: int64

In [26]:
# Create subset of customer data where 'customer_activity' == 'normal' (5 or more orders)

active_cust_ords_prods = cust_ords_prods.loc[cust_ords_prods['customer_activity'] == 'normal']

In [27]:
# Check value counts for customer_activity column

active_cust_ords_prods['customer_activity'].value_counts(dropna=False)

customer_activity
normal    30964564
Name: count, dtype: int64

In [28]:
# Create subset of customer data where 'customer_activity' == 'low' (less than 5 orders)

low_activity_cust_ords_prods = cust_ords_prods.loc[cust_ords_prods['customer_activity'] == 'low']

In [29]:
# Check value counts for customer_activity column

low_activity_cust_ords_prods['customer_activity'].value_counts(dropna=False)

customer_activity
low    1440295
Name: count, dtype: int64

In [30]:
# Export subset of customer data where customer activity is low

# Export low_activity_cust_ords_prods as pickle file low_activity_customers_orders_products.pkl

low_activity_cust_ords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'low_activity_customers_orders_products.pkl'))

#### 5. The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_the_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)


In [31]:
# Using active_cust_ords_prods as dataframe for future analysis

In [32]:
# Create profiling variable based on age

# Look at customer age column characterisitics

active_cust_ords_prods['customer_age'].describe()

count    3.096456e+07
mean     4.946803e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: customer_age, dtype: float64

In [33]:
# Create 4 customer age categories 'age_group'

In [34]:
# If customer age is between 18-33, set value of age group to '18-33'

active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=18) & (active_cust_ords_prods['customer_age'] <= 33), 'age_group'] = '18-33'

/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/3829330634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=18) & (active_cust_ords_prods['customer_age'] <= 33), 'age_group'] = '18-33'
/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/3829330634.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '18-33' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=18) & (active_cust_ords_prods['customer_age'] <= 33), 'age_group'] = '18-33'


In [35]:
# If customer age is between 34-49, set value of age group to '34-49'

active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=34) & (active_cust_ords_prods['customer_age'] <= 49), 'age_group'] = '34-49'

In [36]:
# If customer age is between 50-65, set value of age group to '50-65'

active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=50) & (active_cust_ords_prods['customer_age'] <= 65), 'age_group'] = '50-65'

In [37]:
# If customer age is between 66-81, set value of age group to '66-81'

active_cust_ords_prods.loc[(active_cust_ords_prods['customer_age'] >=66) & (active_cust_ords_prods['customer_age'] <= 81), 'age_group'] = '66-81'

In [38]:
# Check value counts for age_group

active_cust_ords_prods['age_group'].value_counts(dropna=False)

age_group
34-49    7790824
18-33    7757465
66-81    7730834
50-65    7685441
Name: count, dtype: int64

In [39]:
# Create profiling variable based on customer income

# Look at customer income column characterisitics

active_cust_ords_prods['customer_income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: customer_income, dtype: float64

In [40]:
# Create customer income category to designate customer income as low(min-25%), moderate(25-75%), and high(75%-max)

In [41]:
# If customer income is 67,292 or less, set value of income group to 'low'

active_cust_ords_prods.loc[active_cust_ords_prods['customer_income'] <= 67292, 'income_group'] = 'low'

/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/2112519291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_cust_ords_prods.loc[active_cust_ords_prods['customer_income'] <= 67292, 'income_group'] = 'low'
/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/2112519291.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'low' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  active_cust_ords_prods.loc[active_cust_ords_prods['customer_income'] <= 67292, 'income_group'] = 'low'


In [42]:
# If customer income is more than 67,292 and less than 128,102, set value of income group to 'moderate'

active_cust_ords_prods.loc[(active_cust_ords_prods['customer_income'] > 67292) & (active_cust_ords_prods['customer_income'] < 128102), 'income_group'] = 'moderate'

In [43]:
# If customer income is 128,102 or more, set value of income group to 'high'

active_cust_ords_prods.loc[active_cust_ords_prods['customer_income'] >= 128102, 'income_group'] = 'high'

In [44]:
# Check value counts for income_group

active_cust_ords_prods['income_group'].value_counts(dropna=False)

income_group
moderate    15481955
low          7741348
high         7741261
Name: count, dtype: int64

In [45]:
# Create profiling variable based on department

# Look at department column characterisitics

In [46]:
active_cust_ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state',
       'customer_region', 'customer_age', 'date_joined',
       'customer_number_of_dependents', 'customer_family_status',
       'customer_income', 'customer_activity', 'age_group', 'income_group'],
      dtype='object')

In [47]:
active_cust_ords_prods['department_id'].value_counts(dropna=False)

department_id
4     9079273
16    5177182
19    2766406
7     2571901
1     2121731
13    1782705
3     1120828
15    1012074
20    1003834
9      822136
17     699857
12     674781
14     670850
11     424306
18     410392
6      255991
5      144627
8       93060
21      64768
2       34411
10      33451
Name: count, dtype: int64

In [48]:
departments = departments.rename(columns={'Unnamed: 0': 'department_id'})

In [49]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [50]:
# Merge department name into active_cust_ords_prods

active_cust_ords_prods = active_cust_ords_prods.merge(departments, on='department_id', how='left')

In [51]:
active_cust_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,customer_region,customer_age,date_joined,customer_number_of_dependents,customer_family_status,customer_income,customer_activity,age_group,income_group,department
0,2539329,1,1,2,8,NaN,True,196,1,0,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,beverages
1,2398795,1,2,3,7,15.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,beverages
2,473747,1,3,3,12,21.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,beverages
3,2254736,1,4,4,7,29.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,beverages
4,431534,1,5,4,15,28.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,beverages


In [52]:
active_cust_ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state',
       'customer_region', 'customer_age', 'date_joined',
       'customer_number_of_dependents', 'customer_family_status',
       'customer_income', 'customer_activity', 'age_group', 'income_group',
       'department'],
      dtype='object')

In [53]:
# Reorder columns for customer state and customer region to be next to each other

active_cust_ords_prods = active_cust_ords_prods[['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'department', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state',
       'customer_region', 'customer_age', 'date_joined',
       'customer_number_of_dependents', 'customer_family_status',
       'customer_income', 'customer_activity', 'age_group', 'income_group']]

In [54]:
active_cust_ords_prods['department'].value_counts(dropna=False)

department
produce            9079273
dairy eggs         5177182
snacks             2766406
beverages          2571901
frozen             2121731
pantry             1782705
bakery             1120828
canned goods       1012074
deli               1003834
dry goods pasta     822136
household           699857
meat seafood        674781
breakfast           670850
personal care       424306
babies              410392
international       255991
alcohol             144627
pets                 93060
missing              64768
other                34411
bulk                 33451
Name: count, dtype: int64

In [55]:
# Create profiling variable based on number of dependents

In [56]:
active_cust_ords_prods['customer_number_of_dependents'].value_counts(dropna=False)

customer_number_of_dependents
3    7772516
0    7739681
2    7733261
1    7719106
Name: count, dtype: int64

In [57]:
# Flag customers who do not have any dependents as 'no dependents'

active_cust_ords_prods.loc[active_cust_ords_prods['customer_number_of_dependents'] == 0, 'customer_dependents'] = 'no dependents'

/var/folders/rh/wj0j2y0n7j1_m28h5wbms8q00000gn/T/ipykernel_18678/765050542.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'no dependents' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  active_cust_ords_prods.loc[active_cust_ords_prods['customer_number_of_dependents'] == 0, 'customer_dependents'] = 'no dependents'


In [58]:
# Flag customers who have dependents as '1 or more dependents'

active_cust_ords_prods.loc[active_cust_ords_prods['customer_number_of_dependents'] > 0, 'customer_dependents'] = '1 or more dependents'

In [59]:
active_cust_ords_prods['customer_dependents'].value_counts(dropna=False)

customer_dependents
1 or more dependents    23224883
no dependents            7739681
Name: count, dtype: int64

In [60]:
active_cust_ords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'department', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_item_price', 'spending_flag', 'median_days_since_prior_order',
       'frequency_flag', 'customer_gender', 'customer_state',
       'customer_region', 'customer_age', 'date_joined',
       'customer_number_of_dependents', 'customer_family_status',
       'customer_income', 'customer_activity', 'age_group', 'income_group',
       'customer_dependents'],
      dtype='object')

In [61]:
active_cust_ords_prods['customer_family_status'].value_counts(dropna=False)

customer_family_status
married                             21743711
single                               5094410
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: count, dtype: int64

In [62]:
active_cust_ords_prods['customer_dependents'].value_counts(dropna=False)

customer_dependents
1 or more dependents    23224883
no dependents            7739681
Name: count, dtype: int64

In [63]:
active_cust_ords_prods['income_group'].value_counts(dropna=False)

income_group
moderate    15481955
low          7741348
high         7741261
Name: count, dtype: int64

In [64]:
active_cust_ords_prods['age_group'].value_counts(dropna=False)

age_group
34-49    7790824
18-33    7757465
66-81    7730834
50-65    7685441
Name: count, dtype: int64

In [65]:
active_cust_ords_prods['customer_gender'].value_counts(dropna=False)

customer_gender
Male      15586740
Female    15377824
Name: count, dtype: int64

In [70]:
active_cust_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,customer_region,customer_age,date_joined,customer_number_of_dependents,customer_family_status,customer_income,customer_activity,age_group,income_group,customer_dependents
0,2539329,1,1,2,8,NaN,True,196,1,0,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,1 or more dependents
1,2398795,1,2,3,7,15.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,1 or more dependents
2,473747,1,3,3,12,21.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,1 or more dependents
3,2254736,1,4,4,7,29.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,1 or more dependents
4,431534,1,5,4,15,28.0,False,196,1,1,...,South,31,2/17/2019,3,married,40423,normal,18-33,low,1 or more dependents


In [71]:
# Create a new column 'diet' and initialize it as 'none'
active_cust_ords_prods['customer_diet'] = 'none'

In [72]:
# Loop through unique user_ids
unique_user_ids = active_cust_ords_prods['user_id'].unique()
for user_id in unique_user_ids:
    # Create a subset of orders for the current user_id
    user_subset = active_cust_ords_prods[active_cust_ords_prods['user_id'] == user_id]
    
    # Check if 'meat seafood' and 'dairy eggs' are not present in any department in the subset
    if ('meat seafood' not in user_subset['department'].values) and ('dairy eggs' not in user_subset['department'].values):
        # Update the 'diet' column for this user_id to 'vegan'
        active_cust_ords_prods.loc[active_cust_ords_prods['user_id'] == user_id, 'customer_diet'] = 'vegan'
    # Check if 'meat seafood' is not present but 'dairy' is
    elif ('meat seafood' not in user_subset['department'].values) and ('dairy eggs' in user_subset['department'].values):
        # Update the 'diet' column for this user_id to 'vegetarian'
        active_cust_ords_prods.loc[active_cust_ords_prods['user_id'] == user_id, 'customer_diet'] = 'vegetarian'

In [82]:
active_cust_ords_prods['customer_diet'].value_counts(dropna=False)

customer_diet
none          23065332
vegetarian     7589236
vegan           309996
Name: count, dtype: int64

## Export data

In [83]:
# Export active_cust_ords_prods to pickle file

active_cust_ords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'active_customers_orders_products.pkl'))

In [ ]:
# Jordan's code

dict1 = {}
for customer in df["customer"]:
    print(customer)
    print(df[df['customer'] == customer]["department"])
    if 'dairy' in df[df['customer'] == customer]["department"].to_list():
        dict1[customer] = 'dairy'
        print('dairy')
    else:
        dict1[customer] = 'non_dairy'
        print('non_dairy')